In [1]:
import os
import shutil

Data Trimming 

In [3]:
import os
import shutil

def create_debug_subset_sequential(source_dir, dest_dir, train_limit=2000, test_limit=600):
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    os.makedirs(dest_dir, exist_ok=True)

    for split, limit in [("Train", train_limit), ("Test", test_limit)]:
        src_split_path = os.path.join(source_dir, split)
        dst_split_path = os.path.join(dest_dir, split)
        os.makedirs(dst_split_path, exist_ok=True)

        for class_name in os.listdir(src_split_path):
            class_src = os.path.join(src_split_path, class_name)
            class_dst = os.path.join(dst_split_path, class_name)
            os.makedirs(class_dst, exist_ok=True)

            valid_images = sorted([f for f in os.listdir(class_src) if f.endswith(".png") and not f.startswith("._")])
            selected_images = valid_images[:limit]

            for img in selected_images:
                shutil.copy(os.path.join(class_src, img), os.path.join(class_dst, img))

# Correct folder name used here
create_debug_subset_sequential("data_trimmed", "data_trimmed_debug", train_limit=2000, test_limit=600)


In [8]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm


In [2]:
from tensorflow.keras.utils import to_categorical
from transformers import AutoImageProcessor

## Backbone

In [3]:
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

c:\Users\Dell\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--facebook--dinov2-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be t

-Looks inside the main folder to find different categories  

-Gives each category a number so the computer can understand it.

-Goes into each category folder and groups the images that belong to the same video.

-Skips videos that don’t have enough images.

-Breaks the video into small parts (like clips of 8 images).

-Opens and resizes each image so they’re all the same size.

-Processes the images to turn them into numbers the computer can use.

-Averages the 8 images in each clip to make one final image that represents the clip.

-Saves this final image and its category.

-At the end, returns all the images and their categories in a format ready for training a computer model.

In [ ]:
def load_dataset(data_dir, window_size=8):
    X, y = [], []
    class_labels = sorted(os.listdir(data_dir))
    label_map = {cls: idx for idx, cls in enumerate(class_labels)}

    for label in tqdm(class_labels, desc=f"Processing classes from {data_dir}", ncols=100):
        folder_path = os.path.join(data_dir, label)
        video_dict = {}

        for img_name in sorted(os.listdir(folder_path)):
            if not (img_name.endswith(".png") and not img_name.startswith("._")):
                continue
            vid_id = "_".join(img_name.split("_")[:2])
            video_dict.setdefault(vid_id, []).append(os.path.join(folder_path, img_name))

        for vid, frames in video_dict.items():
            if len(frames) < window_size:
                continue
            for i in range(0, len(frames) - window_size + 1, window_size):
                window = frames[i:i + window_size]
                tensor_batch = []
                for frame_path in window:
                    try:
                        img = Image.open(frame_path).convert("RGB").resize((224, 224))
                        pixel_values = image_processor(images=img, return_tensors="pt").pixel_values[0].numpy()
                        tensor_batch.append(pixel_values)
                    except Exception as e:
                        continue
                if len(tensor_batch) == window_size:
                    tensor_batch = np.array(tensor_batch)                         # (8, 3, 224, 224)
                    tensor_batch = np.transpose(tensor_batch, (0, 2, 3, 1))                # (3, 224, 224)
                                                
                    X.append(tensor_batch)             # 8-frame sequence
                    y.append(label_map[label])

    return np.array(X), to_categorical(y, num_classes=len(class_labels))

In [5]:
train_dir = "data_trimmed_debug/Train"
test_dir = "data_trimmed_debug/Test"

In [9]:
# 🚀 Load data
X_train, y_train = load_dataset(train_dir, window_size=8)
X_test, y_test = load_dataset(test_dir, window_size=8)

Processing classes from data_trimmed_debug/Test: 100%|████████████████| 8/8 [02:00<00:00, 15.00s/it]


In [10]:
# 📊 Check shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1971, 224, 224, 3)
y_train shape: (1971, 8)
X_test shape: (591, 224, 224, 3)
y_test shape: (591, 8)


## Train and compile the Backbone 

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import TimeDistributed, LSTM
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [3]:
sequence_input = Input(shape=(8, 224, 224, 3))  # 8 frames per video